Generate Query

In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-small-finetuned-wikiSQL")

def get_sql(query):
  input_text = "translate English to SQL: %s </s>" % query
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])
  
  return tokenizer.decode(output[0])

query = "How many millions of params there are in HF-hub?"

get_sql(query)

# output: 'SELECT COUNT Params FROM table WHERE Location = HF-hub'

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
C:\Users\Asus EK562WS 11i5\AppData\Roaming\Python\Python311\site-packages\transformers\models\auto\m

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

C:\Users\Asus EK562WS 11i5\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus EK562WS 11i5\.cache\huggingface\hub\models--mrm8488--t5-small-finetuned-wikisql. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


'<pad> SELECT COUNT Params FROM table WHERE Location = HF-hub</s>'

In [7]:
import os
import google.generativeai as genai

genai.configure(api_key="gemini-api-key")

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash-exp",
  generation_config=generation_config,
  system_instruction="Your job is to genrate a sql queries you will only output queries and do not give explination and nothing else based on prompt user gives you will receive the overall structure of of database and table schema you must assume that you have to prompt every required query like from only output two queries like:\n mysql>use database_name\nmysql>select *from <table_name>",
)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "we have a table with in database csbs with columns name,roll_no,marks generate queries to get the name of student with maximum marks\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. Here's how I'll approach this, generating the necessary SQL queries step-by-step:\n\n**1. Connect to the Database:**\n\n```sql\nmysql> use csbs;\n```\n\n**2. Query to find the maximum marks:**\n```sql\nmysql> SELECT MAX(marks) AS max_marks FROM students;\n```\n\n**3.  Query to retrieve the student(s) with maximum marks using subquery:**\n\n```sql\nmysql> SELECT name, roll_no, marks\n       FROM students\n       WHERE marks = (SELECT MAX(marks) FROM students);\n```\n**Explanation:**\n\n*   **`USE csbs;`**: This command selects the `csbs` database, making its tables accessible for querying.\n*   **`SELECT MAX(marks) AS max_marks FROM students;`**: This query finds the maximum value in the `marks` column of the `students` table and assigns it an alias of `max_marks`.\n*   **`SELECT name, roll_no, marks FROM students WHERE marks = (SELECT MAX(marks) FROM students);`**:\n    *   This is the main query. It selects the `name`, `roll_no` and `marks` from the `students` table.\n    *   The `WHERE` clause filters the results.\n    *   The subquery `(SELECT MAX(marks) FROM students)` finds the maximum mark as in step 2.\n    *   The outer query then selects students whose marks match the maximum mark.\n\nThis combined approach ensures we first get the maximum mark, and then find the student with that mark. If multiple students share the maximum mark, this query will return them all.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "we have a table with in database csbs with columns name,roll_no,marks generate queries to get the name of student with maximum marks",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. Here's the breakdown of SQL queries to achieve this, along with explanations:\n\n**1. Connect to the Database:**\n\n```sql\nmysql> USE csbs; \n```\n\n**2. Using a Common Table Expression (CTE) to find the maximum marks and students with max marks:**\n```sql\nmysql> WITH MaxMarks AS (\n    SELECT MAX(marks) AS max_marks\n    FROM students\n),\nMaxMarksStudents AS (\n    SELECT name, roll_no, marks\n    FROM students\n    WHERE marks = (SELECT max_marks FROM MaxMarks)\n)\nSELECT name, roll_no, marks FROM MaxMarksStudents;\n\n```\n\n**Explanation:**\n\n*   **`USE csbs;`**: This command selects the `csbs` database as the active database.\n\n*   **`WITH MaxMarks AS (...)`**: This defines a CTE (Common Table Expression) named `MaxMarks`.\n    *   `SELECT MAX(marks) AS max_marks FROM students`: This query within the CTE finds the highest mark from the `students` table and aliases it as `max_marks`.\n\n*   **`MaxMarksStudents AS (...)`**: This defines another CTE named `MaxMarksStudents`.\n     *   `SELECT name, roll_no, marks FROM students WHERE marks = (SELECT max_marks FROM MaxMarks)`: This selects the `name`, `roll_no` and `marks` from the `students` table where the `marks` are equal to the `max_marks` found by `MaxMarks` CTE.\n\n*   **`SELECT name, roll_no, marks FROM MaxMarksStudents;`**: This selects the `name`, `roll_no` and `marks` of students from the `MaxMarksStudents` CTE. This gives us the name of the student(s) with the maximum marks.\n\nThis approach utilizes CTEs for better readability and organization of the query.\n",
      ],
    },
    {
      "role": "user",
      "parts": [
        "we have a table with in database csbs with columns name,roll_no,marks generate queries to get only the name of student with maximum marks",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Okay, I understand. You want to retrieve only the *name* of the student(s) who have the maximum marks. Here are the SQL queries to achieve that:\n\n**1. Connect to the Database:**\n\n```sql\nmysql> USE csbs;\n```\n\n**2.  Using a Subquery to Find the Maximum Marks and Filter Results:**\n\n```sql\nmysql> SELECT name\n       FROM students\n       WHERE marks = (SELECT MAX(marks) FROM students);\n```\n\n**Explanation:**\n\n*   **`USE csbs;`**: This command selects the `csbs` database.\n*   **`SELECT name FROM students ...`**: This part of the query specifies that we only want to retrieve the `name` column from the `students` table.\n*   **`WHERE marks = (SELECT MAX(marks) FROM students)`**:\n    *   This `WHERE` clause filters the rows to include only the student(s) whose `marks` column matches the maximum marks found by the subquery.\n    *   The subquery `(SELECT MAX(marks) FROM students)` finds the highest marks.\n*   The final result will be the `name` of student(s) with the maximum marks.\n\nThis approach efficiently retrieves only the names, avoiding unnecessary data like `roll_no`. If multiple students have the same maximum marks, the query will return all their names.\n",
      ],
    },
  ]
)

response = chat_session.send_message("generate sql queries to find the name of studnet with highest marks")

print(response.text)

```sql
mysql> USE csbs;
```

```sql
mysql> SELECT name FROM students ORDER BY marks DESC LIMIT 1;
```

